In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

%matplotlib inline

/var/folders/t2/qcdrdtq16v1d93n5hnwd7lkw0000gn/T/ipykernel_27827/1424333365.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Define paths
train_dir = "dataset/train"
test_dir = "dataset/test"

# Parameters
batch_size = 64
img_height = 48
img_width = 48

# Load train dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="int",
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True
)

# Load test dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels="inferred",
    label_mode="int",
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=False
)

# Normalize the pixel values
normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.


In [3]:
def dual_path_cnn(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    # Main path
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Residual path
    residual = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(inputs)
    residual = layers.BatchNormalization()(residual)
    residual = layers.MaxPooling2D((2, 2))(residual)  # Apply max pooling to match dimensions

    # Combine paths
    combined = layers.Add()([x, residual])
    combined = layers.ReLU()(combined)

    # Further layers
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(combined)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Flatten and dense layers
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

input_shape = (img_height, img_width, 1)  # Grayscale images
num_classes = 7
model = dual_path_cnn(input_shape, num_classes)

In [4]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=50,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

# Save the model
model.save('fer_ducnn_model.h5')

Epoch 1/50
449/449 [==============================] - 67s 149ms/step - loss: 2.0302 - accuracy: 0.2432 - val_loss: 3.1401 - val_accuracy: 0.2487
Epoch 2/50
449/449 [==============================] - 70s 155ms/step - loss: 1.8129 - accuracy: 0.2501 - val_loss: 1.7337 - val_accuracy: 0.2469
Epoch 3/50
449/449 [==============================] - 73s 162ms/step - loss: 1.7519 - accuracy: 0.2511 - val_loss: 1.6943 - val_accuracy: 0.2834
Epoch 4/50
449/449 [==============================] - 71s 158ms/step - loss: 1.7219 - accuracy: 0.2671 - val_loss: 1.7039 - val_accuracy: 0.3172
Epoch 5/50
449/449 [==============================] - 70s 156ms/step - loss: 1.6967 - accuracy: 0.2805 - val_loss: 1.6647 - val_accuracy: 0.3044
Epoch 6/50
449/449 [==============================] - 67s 149ms/step - loss: 1.6869 - accuracy: 0.2814 - val_loss: 1.6529 - val_accuracy: 0.3206
Epoch 7/50
449/449 [==============================] - 74s 164ms/step - loss: 1.6684 - accuracy: 0.2885 - val_loss: 1.7065 - val_ac

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(test_ds)
print(f'Validation accuracy: {val_accuracy:.2f}')

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')

plt.show()